In [ ]:
import numpy as np
import pandas as pd
from array import *
from PIL import Image, ImageOps
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from numpy import linalg as lg
from skimage.util import *
from skimage.metrics import *
from skimage.filters import laplace, sobel, roberts
from skimage import color, data, measure
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import image
import tensorflow as tf
from tensorboard.plugins import projector
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, Layer, Activation, UpSampling2D,  \
    BatchNormalization, MaxPooling2D, Conv2DTranspose, Reshape, BatchNormalization, ReLU, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras import activations, callbacks
import random
import cv2
import math
import os
from os.path import isfile, join, exists
import re
import joblib
import glob
import json
from collections import Counter
import time

sys.path.append('..\\..\\..\\utils')
from utils import *
from autoencoder_utils import *

In [ ]:
constants_dictionary = json.load(open("..\\..\\..\\utils\\constants.json","r"))
VALID_NOISE_STRENGHTS = constants_dictionary.get('VALID_NOISE_STRENGHTS')
VALID_NOISE_TYPES = constants_dictionary.get('VALID_NOISE_TYPES') 
noise_dictionary = constants_dictionary.get('noise_dictionary')
noise_levels_dictionary = constants_dictionary.get('noise_levels_dictionary')
categorical_classes_dictionary = constants_dictionary.get('categorical_classes_dictionary')
base_name = constants_dictionary.get('base_name_dae_images')
mypath = constants_dictionary.get('path_images')


# Denoising Autoencoder Grayscale for images 256x256

## Grayscale images of size 256x256

In [ ]:
# Denoising Autoencoder Grayscale

images_size_256= 256
x_256, y_256, y_noise_info_256 = get_ds_from_pkl(base_name, resize_images_to=images_size_256)

X_train_256, X_test_256, y_train_256, y_test_256 = train_test_split(
    x_256, 
    y_256, 
    test_size=0.15, 
    shuffle=True,
    random_state=42,
)
d4_train_256 = X_train_256.reshape((X_train_256.shape[0], X_train_256.shape[1], X_train_256.shape[2], 1))
d4_test_256=X_test_256.reshape((X_test_256.shape[0], X_test_256.shape[1], X_test_256.shape[2], 1))
print('shape of training datset: ', d4_train_256.shape)

In [ ]:
try:
    dae_grayscale = load_model('..\\..\\..\\checkpoints\\Denoising_Autoencoder_grayscale.h5')
except:
    dae_grayscale = create_model('Denoising_Autoencoder_grayscale')
    dae_grayscale.compile(optimizer="adam", loss="mse")
    dae_grayscale.summary()
    checkpoint = ModelCheckpoint('..\\..\\..\\checkpoints\\Denoising_Autoencoder_grayscale.h5', save_best_only=True, save_weights_only=False, verbose=1)
    history = dae_grayscale.fit(d4_train_256, y_train_256, batch_size=4, epochs=50, callbacks=checkpoint, validation_split=0.25, verbose=2)


## Evaluation

### PSNR and blurrines

In [ ]:
evaluate_psnr_blur(d4_train_256,y_train_256,dae_grayscale)
evaluate_psnr_blur(d4_test_256, y_test_256, dae_grayscale)


In [ ]:
# evaluate each level of noise (levels from 1-8) and each category of noise level (low/medium/high)
for var in noise_levels_dictionary.keys() + noise_dictionary.keys():
    evaluate_image('gaussian', var, dae_grayscale)
    evaluate_image('s&p', var, dae_grayscale)    
    evaluate_image('speckle', var, dae_grayscale)   


###  execution ime



In [ ]:
start_time = time.time()
x_test, y_test, y_ground_truth = get_single_noisy_image('gaussian', 'level5')
prediction = dae_grayscale.predict(np.expand_dims((x_test), axis=0))
end_time = time.time()
elapsed_time = end_time - start_time
print('Execution time:', elapsed_time, 'seconds')


# Grid Search

In [ ]:
gridSearch_X = np.reshape(d4_dae_noise_test_dataset_gr_64, (d4_dae_noise_test_dataset_gr_64.shape[0], 64*64))
gridSearch_Y = np.reshape(y_dae_noise_test_gray_64, (y_dae_noise_test_gray_64.shape[0], 64*64))

In [ ]:
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# define the grid search parameters
batch_size = [2, 4, 8, 16, 32]
epochs = [10, 40, 60, 100]
lr =  [0.0001, 0.001]
opt = ["adam", "sgd"]
param_grid = dict(batch_size=batch_size, epochs=epochs,optimizer__learning_rate=lr, optimizer=opt)
model = KerasRegressor(model=dae, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(gridSearch_X, gridSearch_Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
grid.get_params().keys()

In [ ]:
print("Best: %f using %s" % (gs.best_score_, gs.best_params_))

## Grayscale images of size 512x512

In [ ]:
images_size_512= 512
x_512, y_512, y_noise_info_512 = get_ds_from_pkl(base_name, resize_images_to=images_size_512)

X_train_512, X_test_512, y_train_512, y_test_512 = train_test_split(
    x_512, 
    y_512, 
    test_size=0.15, 
    shuffle=True,
    random_state=42,
)
d4_train_512 = X_train_512.reshape((X_train_512.shape[0], X_train_512.shape[1], X_train_512.shape[2], 1))
d4_test_512=X_test_512.reshape((X_test_512.shape[0], X_test_512.shape[1], X_test_512.shape[2], 1))


## Grayscale images of size 64x64

In [ ]:
images_size_64=64
load_mixed_ds_to_pkl(resize_to=images_size_64)
x_64, y_64, y_noise_info_64 = get_ds_from_pkl(base_name, resize_images_to=images_size_64)

X_train_64, X_test_64, y_train_64, y_test_64 = train_test_split(
    x_64, 
    y_64, 
    test_size=0.15, 
    shuffle=True,
    random_state=42,
)
d4_train_64 = X_train_64.reshape((X_train_64.shape[0], X_train_64.shape[1], X_train_64.shape[2], 1))
d4_test_64=X_test_64.reshape((X_test_64.shape[0], X_test_64.shape[1], X_test_64.shape[2], 1))



# Denoising Autoencoder Grayscale for images with arbitrary size

In [ ]:
# from https://github.com/developershutt/Autoencoders/blcob/main/3%20-%20Denoise%20Autoencoder/Code.ipynb

# Encoder
encoder_input = Input(shape=(None,None,1))
x1 = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_input)
x1 = BatchNormalization()(x1)
x1 = MaxPool2D(pool_size = (2,2), padding='same')(x1)
x2 = Conv2D(64, (3,3), activation='relu', padding='same')(x1)
x2 = BatchNormalization()(x2)
x2 = MaxPool2D(pool_size = (2,2), padding='same')(x2)
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(x2)
x3 = BatchNormalization()(x3)
encoded = MaxPool2D(pool_size = (2,2), padding='same')(x3)


# Decoder
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(encoded)
x3 = BatchNormalization()(x3)
x3 = UpSampling2D((2,2))(x3)
x2 = Conv2D(64, (3,3), activation='relu', padding='same')(x2)
x2 = BatchNormalization()(x2)
x2 = UpSampling2D((2,2))(x2)
x1 = Conv2D(128, (3,3), activation='relu', padding='same')(x2)
x1 = BatchNormalization()(x1)
x1 = UpSampling2D((2,2))(x1)
decoded = Conv2D(1, (3,3), activation='sigmoid', padding= 'same')(x1)

dae_simple = Model(encoder_input, decoded, name = 'Denoising_Autoencoder_grayscale_uniform')
dae_simple.compile(optimizer="adam", loss="binary_crossentropy")
dae_simple.summary()

In [ ]:
with tf.device('/CPU:0'):
    checkpoint = ModelCheckpoint('Denoising_Autoencoder_grayscale_uniform.h5', save_best_only=True, save_weights_only=False, verbose=1)
    history = dae_simple.fit(d4_train_512, train_gray_512_y, batch_size=1, epochs=40, callbacks=checkpoint, validation_split=0.25, verbose=2)

## Evaluation

In [ ]:
dae_grayscale = load_model('Denoising_Autoencoder_grayscale_uniform.h5')
dae_grayscale.summary()

In [ ]:
visualize_data(d4_train_64[20:23], 1,3)
visualize_data(d4_train_64[20:23], 1,3)
prediction_gray = dae_grayscale.predict(d4_train_64[20:23], batch_size=1)
visualize_data(prediction_gray, 1,3)

evaluate_psnr_blur(d4_train_64,y_train_512,dae_grayscale)
evaluate_psnr_blur(d4_test_512, y_test_512, dae_grayscale)

for var in noise_levels_dictionary.keys():
    evaluate_image('gaussian', var, dae_grayscale)
    evaluate_image('s&p', var, dae_grayscale)    
    evaluate_image('speckle', var, dae_grayscale)   
    
# evaluate each category of noise level (low/medium/high)
for var in noise_dictionary.keys():
    evaluate_image('gaussian', var, dae_grayscale)
    evaluate_image('s&p', var, dae_grayscale)    
    evaluate_image('speckle', var, dae_grayscale)

# DENOISING AUTOENCODER RGB

In [ ]:
# Encoder

encoder_input = Input(shape=(None,None,3))
x1 = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_input)
x1 = BatchNormalization()(x1)
x1 = MaxPool2D(pool_size = (2,2), padding='same')(x1)
x2 = Conv2D(64, (3,3), activation='relu', padding='same')(x1)
x2 = BatchNormalization()(x2)
x2 = MaxPool2D(pool_size = (2,2), padding='same')(x2)
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(x2)
x3 = BatchNormalization()(x3)
encoded = MaxPool2D(pool_size = (2,2), padding='same')(x3)


# Decoder
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(encoded)
x3 = BatchNormalization()(x3)
x3 = UpSampling2D((2,2))(x3)
x2 = Conv2D(64, (3,3), activation='relu', padding='same')(x2)
x2 = BatchNormalization()(x2)
x2 = UpSampling2D((2,2))(x2)
x1 = Conv2D(128, (3,3), activation='relu', padding='same')(x2)
x1 = BatchNormalization()(x1)
x1 = UpSampling2D((2,2))(x1)
decoded = Conv2D(3, (3,3), activation='sigmoid', padding= 'same')(x1)

dae_rgb = Model(encoder_input, decoded, name = 'Denoising_Autoencoder_rgb')
dae_rgb.compile(optimizer="adam", loss="binary_crossentropy")
dae_rgb.summary()

In [ ]:
with tf.device('/CPU:0'):
    checkpoint = ModelCheckpoint('Denoising_Autoencoder_rgb.h5', save_best_only=True, save_weights_only=False, verbose=1)
    history = dae_rgb.fit(d4_rgb_256_train_dataset, y_rgb_256_train, batch_size=1, epochs=40, callbacks=checkpoint, validation_split=0.25, verbose=2)

In [ ]:
dae_rgb = load_model('Denoising_Autoencoder_rgb.h5')
dae_rgb.summary()

# Evaluate

In [ ]:
#Testing

visualize_data(d4_rgb_256_test_dataset[:3], 1,3, mode='rgb')
pred_test_rgb_256 = dae_rgb.predict(d4_rgb_256_test_dataset[:3], batch_size=1)
visualize_data(pred_test_rgb_256, 1, 3, mode='rgb')
visualize_data(y_rgb_256_test[:3], 1, 3, mode='rgb')

evaluate_psnr_blur(d4_train_256_rgb,y_train_256_rgb,dae_rgb)
evaluate_psnr_blur(d4_rgb_256_test_dataset, y_rgb_256_test, dae_rgb)

for var in noise_levels_dictionary.keys():
    evaluate_image('gaussian', var, dae_rgb)
    evaluate_image('s&p', var, dae_rgb)    
    evaluate_image('speckle', var, dae_rgb)   
    
# evaluate each category of noise level (low/medium/high)
for var in noise_dictionary.keys():
    evaluate_image('gaussian', var, dae_rgb)
    evaluate_image('s&p', var, dae_rgb)    
    evaluate_image('speckle', var, dae_rgb)

#  RGB images

## load 512 & 256

In [ ]:
x_512, y_512, y_noise_info_512 = get_ds_from_pkl(base_name, resize_images_to=images_size_512, mode='rgb')

X_train_512_rgb, X_test_512_rgb, y_train_512_rgb, y_test_512_rgb = train_test_split(
    x_512, 
    y_512, 
    test_size=0.15, 
    shuffle=True,
    random_state=42,
)
d4_train_512_rgb = X_train_512_rgb.reshape((X_train_512_rgb.shape[0], X_train_512_rgb.shape[1], X_train_512_rgb.shape[2], 1))
d4_test_512_rgb=X_test_512_rgb.reshape((X_test_512_rgb.shape[0], X_test_512_rgb.shape[1], X_test_512_rgb.shape[2], 1))

x_256, y_256, y_noise_info_256 = get_ds_from_pkl(base_name, resize_images_to=images_size_256, mode='rgb')

X_train_256_rgb, X_test_256_rgb, y_train_256_rgb, y_test_256_rgb = train_test_split(
    x_256, 
    y_256, 
    test_size=0.15, 
    shuffle=True,
    random_state=42,
)
d4_train_256_rgb = X_train_256_rgb.reshape((X_train_256_rgb.shape[0], X_train_256_rgb.shape[1], X_train_256_rgb.shape[2], 1))
d4_test_256_rgb=X_test_256_rgb.reshape((X_test_256_rgb.shape[0], X_test_256_rgb.shape[1], X_test_256_rgb.shape[2], 1))
